In [ ]:
%env VECLIB_MAXIMUM_THREADS=1
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

!make clean

import numpy as np
#np.show_config()

# Cache optimization

1. Memory hierarchy
2. How cache works
3. Stride analysis
4. Tiling

# Memory hierarchy

In the simplest model of computers, we write programs that process data in a linearly-addressed memory.  In reality, to achieve good runtime performance, the memory is hierarchical.  The programs we write still treat it like a linearly-addressed space, but by understanding the underneath hierarchical structure, we may get the most performance from the system.

When it comes to performance, we may be tempted to make all memory as fast as possible.  It is impractical because fast memory is very difficult to manufacture.  The economocally feasible approach is to keep the most frequently used data in the fastest memory, and the infrequently used data in slow and inexpensive memory.  When the work with the data is finished, they are removed from the fast memory, and other data are loaded to it.

Memory comes in many kinds in the hierarchy.  I put them in 4 categories:

1. CPU register file

   The registers reside in the CPU circuits.  Instructions (machine code or assembly) can directly operate them and the electronic signals flow through the CPU circuits.  There is no delay in access time to registers.  They are the fastest memory.
2. CPU cache

   CPU cache memory works as a 'buffer' between the registers and the main memory.  It usually uses fast and expensive static random access memory (SRAM).  It is called static because the circuit keeps in one of the two stable states and access doesn't change the state.  The circuit takes more transistors than slower types of memory.

   The cache memory may be part of the CPU circuit or outside it.  A CPU usually has multiple levels of cache.  It can be a couple of MBs, or as large as tens or hundreds of MBs:
   * Intel Xeon Platinum 9282 has 77MB L3 (56 cores): https://en.wikichip.org/wiki/intel/xeon_platinum/9282
   * AMD EPYC 7H12 has 256MB L3 (64 cores): https://www.amd.com/en/products/cpu/amd-epyc-7h12
3. Main memory

   Main memory is away from the CPU chip package.  It usually uses less expensive dynamic random access memory (DRAM).  The circuit takes more time to access the data but the lower cost allows much larger space.

   Data in the main memory are lost when the system is powered off.

   Mainstream PC uses DDR (double data rate) 4 SDRAM (synchronous dynamic random-access memory) with DIMM (dual in-line memory module) and its variants, e.g. RDIMM (registered DIMM) and LRDIMM (load reduced DIMM).  Depending on the CPU memory controller bandwidth, the data throughput may be around 60GB/s or higher.
   
   A powerful server may have up to 6TB of main memory:
   * 24 RDIMM/LRDIMM slots: https://www.supermicro.com/en/products/ultra 
   * 256GB RDIMM/LRDIMM module: https://www.samsung.com/semiconductor/dram/module/
4. Storage

   Data on the storage cannot be directly accessed by CPU instructions.  They need to be loaded to main memory and then the instructions can touch them.  The loading and saving operations are considered input and output (I/O).  Data in the storage are presisted when the system is powered off.

   The storage is usually called the "disks", because it used to be hard-disk drives (HDD).  In a recent system the storage changes to use the solid-state drives (SSD), which use the flash memory instead of hard disk.
   * Seagate Burracuda 510 SSD sequential read 3.45 GB/s, write 3.2 GB/s: https://www.seagate.com/internal-hard-drives/ssd/barracuda-ssd/
   * Samsung PM1735 SSD sequential read 8 GB/s, write 3.8 GB/s: https://www.samsung.com/semiconductor/ssd/enterprise-ssd/

Here is a table (excerpt from Figure 6.23 in CS:APP (3/e)) showing the latency of each of the memory, measured in CPU cycle (less than 1 ns):

| Type | Latency (cycles) |
| -- | -- |
| CPU registers | 0 |
| L1 cache | 4 | 
| L2 cache | 10 |
| L3 cache | 50 |
| Main memory | 200 |
| Storage (disk) | 100,000 |

Nothing is fast without cache.

# How cache works

There are 3 ways to implement caches: (i) direct-map caches, (ii) set-associative caches, and (iii) fully-associative caches.  The direct-map caches are the simplest one and I will use it to show the basic concepts of caches.

If the accessed byte is already in the cache, it's a hit, and CPU directly gets the byte from the cache.  If the byte is not in the cache, it's a miss, and the memory controller goes to the main memory to fetch the byte into cache, before CPU gets it.

According to the previous table, a cache miss is expensive.  When CPU can get data from cache, the latency is around a couple of cycles.  When there is a miss, additional hundreds of cycles are required to get the data.

There are two kinds of misses.  A cold miss happens when the requested byte is not in the cache.  The second kind is conflict miss, and happens when multiple cachable data claim the same cache block.  One set pops out the other, and vice versa, and each access is a miss.

Assume we have a main memory of 64 bytes (6-bit address is sufficient), and a cache of 8 bytes, let's use the following example to show how a cache works:

| Access # | Decimal addr | Binary addr | Hit or miss | Cache block (binary) |
| -- | -- | -- | -- | -- |
| 1 | 22 | 010110 | miss (cold) | 110 |
| 2 | 26 | 011010 | miss (cold) | 010 |
| 3 | 22 | 010110 | hit | 110 |
| 4 | 26 | 011010 | hit | 010 |
| 5 | 16 | 010000 | miss (cold) | 000 |
| 6 | 18 | 010010 | miss (conflict) | 010 |
| 7 | 26 | 011010 | miss (conflict) | 010 |

This is a direct-map cache.  To reduce conflict misses, we may use multi-way set associativity.  2-, 4-, or 8-way set associative cache is commonly used.  Full associativity is too expensive in circuit implementation.

# Memory accessing speed is determined by cache block (line) size

A cache block usually contains more than one byte or word.  In x86, the block (line) size is 64 bytes.  That is, when loading data from main memory to cache, it's done once a block, rather than byte by byte.

```cpp
// Sequential.
for (int i=0; i<nelem; ++i) { arr[i] = i; }
sw.lap();
for (int i=0; i<nelem; ++i) { arr[i] *= 3; }
elapsed = sw.lap();

// Skipping 2.
for (int i=0; i<nelem; ++i) { arr[i] = i; }
sw.lap();
for (int i=0; i<nelem; i+=2) { arr[i] *= 3; }
elapsed = sw.lap();

// ... 4, 8, 16, ... 1024.
```

In [ ]:
# Show how main memory (dram) access dominates runtime.
!make 01_skip_access ; ./01_skip_access

# Locality

While coding we usually don't have a lot of time to do detailed cache analysis.  Instead, we keep in mind that the code runs faster when it's more compact.  This is the concept of locality.

There are two kinds of locality: temporal and spatial.  Temporal locality means that a fixed address is reused in the near future.  Spatial locality means that the addresses close to the current address is reused in the near future.  The better locality, of either kind, improves the performance.  And the cache hierarchy is why locality works.

To take advantage of locality, programmers analyze by using "strides".  A stride is the number of indexes to elements to slide when accessing the data in arrays.  The most basic striding is sequential access, or the 1-stride.  Similarly, we may have n-strides.  The larger the stride is, the worse the locality.

Recall that x86 uses 64-byte cache blocks, and a double-precision floating point takes 8 bytes.

## Demonstrate by matrix population

To demonstrate how the data layout, i.e., majoring or striding, affects runtime, we use an example of populating $1024 \times 1024 \times 64$ integer elements as a matrix.  The following shapes are benchmarked (total number of elements remains unchanged):

* $(1024\times1024\times64) \times 1$, i.e., one-dimension
* $(1024\times1024\times32) \times 2$
* $(1024\times1024\times16) \times 4$
* $(1024\times1024\times8) \times 8$
* $(1024\times1024\times4) \times 16$
* $(1024\times1024\times2) \times 32$
* $(1024\times1024) \times 64$
* $(1024\times512) \times 128$
* $(1024\times256) \times 256$
* $(1024\times128) \times 512$
* $(1024\times64) \times 1024$
* $(1024\times32) \times (1024\times2)$
* $(1024\times16) \times (1024\times4)$
* $(1024\times8) \times (1024\times8)$

We populate the matrices along two axes.  First we iterate over the last index (row):

```cpp
// Populate by last index.
for (size_t i=0; i<nrow; ++i) // the i-th row
{
    for (size_t j=0; j<ncol; ++j) // the j-th column
    {
        buffer[i*ncol + j] = i*ncol + j;
    }
}
```

Then iterate over the first index (column):

```cpp
// Populate by first index.
for (size_t j=0; j<ncol; ++j) // the j-th column
{
    for (size_t i=0; i<nrow; ++i) // the i-th row
    {
        buffer[i*ncol + j] = i*ncol + j;
    }
}
```

We will see the speed is very different.

In [ ]:
# Show how striding affects population runtime.
!make 02_locality ; ./02_locality

While writing programs, it's much easier to know the stride than analyzing the cache behavior.  The latter, in many scenarios, is prohibitively difficult.

Since we know the cache line is 64 byte wide, we expect the cache performance may significantly reduce when the stride is around that value (16 int elements).  As shown in the above benchmark.

### Condition the memory and the cache before benchmarking

```cpp
// Prepopulation to cancel the effect of overcommit or delayed allocation.
for (size_t i=0; i<nelem; ++i) { buffer[i] = nelem-i; }
```

## Array majoring in numpy

Array majoring is directly related to locality.  The difference in the performance of matrix-vector multiplication is show for row- and column-majoring arrays.

In [ ]:
%%time
dim = 10000
float_rmajor = np.arange(dim*dim, dtype='float64').reshape((dim,dim))
float_cmajor = float_rmajor.T.copy().T
vec = np.arange(dim, dtype='float64')

In [ ]:
%%time
res_float_rmajor = np.dot(float_rmajor, vec)

In [ ]:
%%time
res_float_cmajor = np.dot(float_cmajor, vec)

### Integer matrix-vector multiplication

In [ ]:
%%time
dim = 10000
int_rmajor = np.arange(dim*dim, dtype='int64').reshape((dim,dim))
int_cmajor = int_rmajor.T.copy().T
vec = np.arange(dim, dtype='int64')

In [ ]:
%%time
res_int_rmajor = np.dot(int_rmajor, vec)

In [ ]:
%%time
res_int_cmajor = np.dot(int_cmajor, vec)

The performance difference of integer arrays is much larger than floating-point arrays.  Note that `double` and `int64` both take 8 bytes.  Reason: LAPACK / MKL / vecLib.

For the same reason, the floating-point multiplication is slightly faster than the integer.

# Tiling

This is a naive implementation of matrix-matrix multiplication:

```cpp
for (size_t i=0; i<mat1.nrow(); ++i)
{
    for (size_t k=0; k<mat2.ncol(); ++k)
    {
        double v = 0;
        for (size_t j=0; j<mat1.ncol(); ++j)
        {
            v += mat1(i,j) * mat2(j,k);
        }
        ret(i,k) = v;
    }
}
```

The matrices are row-major.  The stride for the second matrix is `ncol2`, so it doesn't have good locality.  This naive implementation is clear, but the performance is bad.

Matrix-matrix multiplication is one of the most important problems for numerical calculation, and there are many techniques available for making it fast.  Most if not all of them are about hiding the memory access latency.  Tiling is a basic technique that delivers impressive speed-up by reordering the calculation and making it cache-friendly.

In [ ]:
# Show how tiling improves runtime performance.
!make 03_matrix_matrix ; ./03_matrix_matrix

# Exercises

1. Consult the data sheet of one x86 CPU and one Arm CPU.  Make a table for the line size of each of the cache levels, and compare the difference between the two CPUs.
2. Write a program that uses tiling to speed up matrix-matrix multiplication, and do not require the matrix dimension to be multiples of the tile size. 

# References

* Computer Systems: A Programmer's Perspective, Chapter 6 The Memory Hierarchy, Randal E. Bryant and David R. O'Hallaron: https://csapp.cs.cmu.edu/
* Gallery of Processor Cache Effects: http://igoro.com/archive/gallery-of-processor-cache-effects/
* Lecture Notes of Applications of Parallel Computers by David Bindel: https://www.cs.cornell.edu/~bindel/class/cs5220-s10/slides/lec03.pdf
* https://en.wikichip.org/wiki/WikiChip
* https://www.uops.info/